algorithms samples

In [ ]:
import random
import numpy as np


# זמן טיפול בספא
def sample_spa_time():
    u = np.random.uniform(0, 1)
    if u < 0.5:
        return 0.5 + np.sqrt(u / 8)
    else:
        return 1 - np.sqrt((1 - u) / 8)


#  זמן שירות בר
def sample_bar_time():
    while True:
        u1, u2 = np.random.uniform(0, 1, 2)
        z = np.sqrt(-2 * np.log(u1)) * np.cos(2 * np.pi * u2)
        X = z * np.sqrt(1.5) + 5
        if X >= 0:
            return X


# זמן הגעה לארוחת בוקר
def sample_breakfast_arrival():
    lower_bound = 6.5
    upper_bound = 11.5
    lmbda = 15
    u = np.random.uniform(0, 1)
    X = -np.log(1 - u * (1 - np.exp(-lmbda * lower_bound))) / lmbda
    if X <= upper_bound:
        return X
    else:
        return sample_breakfast_arrival()


#  זמן ארוחת בוקר
def sample_breakfast_time():
    while True:
        u1, u2 = np.random.uniform(0, 1, 2)
        z = np.sqrt(-2 * np.log(u1)) * np.cos(2 * np.pi * u2)
        X = z * 10 + 40
        if X >= 0:
            return X


 # זמן שהייה בבריכה
def sample_time_at_pool():
    u = random.random()  # This generates a random float uniformly in the range [0.0, 1.0)

    if u < 0.25:
        return 60 * (np.sqrt(12 * u + 1))
    elif u < 0.875:
        return 60 * (-5 + np.sqrt(441 + 640 * u)) / 8
    else:
        return 60 * (3 + 8 * u)


 #זמן שהייה במלון
def sample_length_stay_hotel():
    u = random.Random()
    if u < 0.25:
        return 1  # 1 night with probability of 0.25
    elif u <0.25 + 0.46:
        return 2   #2 nights with probability 0.46
    elif u< 0.25+0.46+0.2:
        return 3  #3 nights with probability 0.2
    elif u < 0.25+0.46+0.2+0.05:
        return 4             #4 night with probablity 0.05
    else:         # 5 nights at probability of 0.04
        return 5


# זמן שירות צק אין
def sample_check_in():
    rate = 0.14
    scale = 1 / rate
    return np.random.exponential(scale)


# זמן שירות צק אאוט
def sample_check_out():
    rate=0.2
    scale = 1 / rate
    return np.random.exponential(scale)


# זמן הגעת אורח למלון
def sample_arrival_guest(hotel):
    rooms_total = 110
    rooms_available = hotel.available_rooms()
    average_rating = hotel.rating
    b1 = 1.5
    b2 = 2
    a = 20
    if rooms_available > 0 :
        lamda = a * ((rooms_available / rooms_total) ** b1) * (average_rating / 10) ** b2
        return np.random.exponential(lamda)
    return None





classes


In [ ]:
class Hotel:
    def __init__(self):
        self.rooms = []  # רשימת חדרים במלון
        self.create_rooms()
        self.rating = 7.0  # דירוג ראשוני של המלון

# יצירת חדרים במלון לפי סוגים וקיבולות מוגדרות מראש
    def create_rooms(self):
        for i in range(1, 31):
            self.rooms.append(Room(room_number=i, room_type="family", capacity=5))
        for i in range(31, 71):
            self.rooms.append(Room(room_number=i, room_type="family", capacity=3))
        for i in range(71, 101):
            self.rooms.append(Room(room_number=i, room_type="couple", capacity=2))
        for i in range(101, 111):
            self.rooms.append(Room(room_number=i, room_type="suite", capacity=2))

#  מחפש חדר פנוי שמתאים לגודל הקבוצה של האורח.
    def find_available_room(self, group_size, group_type, suite):
         for room in self.rooms:
            # אם החדר פנוי ומתאים לקבוצה לפי גודל, סוג, וסוויטה
            if not room.occupied and room.capacity >= group_size:
                if (group_type == 'family' and room.room_type == 'family') or \
                   (group_type == 'couple' and room.room_type == 'couple') or \
                   (group_type == 'single' and room.room_type == 'couple') or \
                   (suite == (room.room_type == 'suite')):
                        return room
         return None

# מוצא את מספר החדרים הפנויים במלון
    def available_rooms(self):
        count = 0
        for room in self.rooms:
            if not room.occupied:
                count += 1
        return count


class Room:
    def __init__(self, room_number, room_type, capacity):
        self.room_number = room_number  # מזהה ייחודי.
        self.room_type = room_type  # סוג החדר.
        self.capacity = capacity  # הקיבולת המרבית.
        self.occupied = False  # האם החדר תפוס כרגע?

#        מסמן את החדר כתפוס כאשר אורח עושה צ'ק-אין.
    def check_in(self):
        if self.occupied:
            raise ValueError(f"Room {self.room_number} is already occupied.")
        self.occupied = True

#        מסמן את החדר כלא תפוס כאשר אורח עושה צ'ק-אאוט.
    def check_out(self):
        if not self.occupied:
            raise ValueError(f"Room {self.room_number} is already vacant.")
        self.occupied = False
        print(f"Room {self.room_number} is now vacant.")



class Group:
    def __init__(self, group_type, group_size, suite):
        self.group_type = group_type  # סוג הקבוצה
        self.group_size = group_size  # מספר אנשים בקבוצה
        self.suite = suite  # האם הקבוצה בסוויטה



events

In [ ]:
import heapq

class Event:
    def __init__(self, time):
        self.time = time

    def __lt__(self, other):
        return self.time < other.time

    def handle(self, simulation):
        print("Handle method must be implemented by subclasses")
        pass


class ArrivalEvent(Event):
    def handle(self, simulation):
        # תזמון הגעה הבאה
        next_arrival_time = self.time + sample_arrival_guest(simulation.hotel)
        if next_arrival_time < simulation.simulation_time:
            simulation.schedule_event(ArrivalEvent(next_arrival_time))
        group = simulation.create_group()
        # חיפוש האם יש חדר פנוי
        room = simulation.hotel.find_available_room(group.group_size, group.group_type, group.suite)
        if not room is None : #אם יש חדר
            room.check_in()
            days = sample_length_stay_hotel()
            for day in range(days):
                start_time = sample_breakfast_arrival() + day*24*60
                simulation.schedule_event(StartDayEvent(start_time)) #לבדוק איך לעשות את זה
                checkout_time = self.time + (days * 24 * 60)
                simulation.schedule_event(CheckOutEvent(checkout_time, room))
            if simulation.is_recepionist_available():
                simulation.assign_to_reception(self.time)
            else:
                simulation.reception_queue.append(self.time)







class HotelSimulation:
    """
    מחלקה לניהול סימולציית המלון.
    """

    def __init__(self, simulation_time):
        self.hotel = Hotel()

        # Events and metrics
        self.event_queue = []

       # initialize simulation
        self.simulation_time = simulation_time

       #employees
        self.num_receptionists = 2

       #reception state
        self.receptionists_busy = 0
        self.reception_queue = []

    def schedule_event(self, event):
        heapq.heappush(self.event_queue, event)

    def create_group(self):
        # בחירת סוג קבוצה: משפחה, זוג או יחיד
        group_type = np.random.choice(['family', 'couple', 'single'], p=[0.4, 0.4, 0.2])
        # הגדרת מספר אנשים בקבוצה לפי סוג
        if group_type == 'family':
            group_size = np.random.choice([3, 4, 5], p=[0.3, 0.2, 0.5])
        elif group_type == 'couple':
            group_size = 2
        else:  # 'single'
            group_size = 1
        # הגדרת האם הקבוצה בסוויטה
        suite = np.random.random() < 0.1 and group_type in ['couple', 'single']
        # יצירת והחזרת אובייקט קבוצה
        return Group(group_type, group_size, suite)

    def is_recepionist_available(self):
        return self.receptionists_busy < self.num_receptionists

    def assign_to_reception(self, time):
        self.receptionists_busy += 1
        service_time = sample_check_in()
        self.schedule_event(CheckInEvent(time + service_time))



    def run(self):

        # Schedule the first arrival
        self.schedule_event(ArrivalEvent(0))

        # Process events
        while self.event_queue and self.event_queue[0].time < self.simulation_time:
            event = heapq.heappop(self.event_queue)
            event.handle(self)

# Simulation Parameters


# Run simulation
simulation = HotelSimulation(120)
simulation.run()